In [2]:
import numpy as np
from matplotlib import pyplot as plt
import utils
from scipy import signal
import mne


In [11]:
data_preprocessed = mne.io.read_raw_edf(r'A:\Sano\preprocessed_adult\chb21\chb21_19.edf', preload=False, verbose=False)
data_raw = utils.load_and_dump_channels(r'A:\Sano\adult_data\chb21\chb21_19.edf')

Removing channel --0
Removing channel --1
Removing channel --2
Removing channel --3
Removing channel --4
Removing channel T8-P8-1


a:\Sano\utils.py:172: RuntimeWarning: Channel names are not unique, found duplicates for: {'-', 'T8-P8'}. Applying running numbers for duplicates.
  data_raw = mne.io.read_raw_edf(filepath, preload=False, verbose=False)
a:\Sano\utils.py:172: RuntimeWarning: Scaling factor is not defined in following channels:
--0, --1, --2, --3, --4
  data_raw = mne.io.read_raw_edf(filepath, preload=False, verbose=False)


In [5]:
%matplotlib qt

In [ ]:
adult_data =r'A:\Sano\adult_data'
peprocessed_data = r'A:\Sano\preprocessed_adult'
subject_seizures = r'A:\Sano\adult_data\RECORDS-WITH-SEIZURES'

In [3]:
utils.preprocess_dataset(subject_seizures,adult_data,peprocessed_data)

Finished preprocessing subject chb23/chb23_09.edf.
Subject chb24/chb24_01.edf not found.
Subject chb24/chb24_03.edf not found.
Subject chb24/chb24_04.edf not found.
Subject chb24/chb24_06.edf not found.
Subject chb24/chb24_07.edf not found.
Subject chb24/chb24_09.edf not found.
Subject chb24/chb24_11.edf not found.
Subject chb24/chb24_13.edf not found.
Subject chb24/chb24_14.edf not found.
Subject chb24/chb24_15.edf not found.
Subject chb24/chb24_17.edf not found.
Subject chb24/chb24_21.edf not found.


In [ ]:
data_raw =utils.load_and_dump_channels(r"A:\Sano\adult_data\chb23\chb23_09.edf")
utils.reorder_channels_chbmit(data_raw)
raw_copy = utils.run_preprocessing(data_raw,18,apply_ica=False,apply_pca = True,freq_l=2)